# Spectra acquisiton
Control both the stepping motor and the camera from this notebook.<br>
Make a loop to measure a wide-range spectrum automatically.

NOTE: only one connection to COM port is possible at a time. Shut down other connections, either in python or dedicated software.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
import time, os, logging
from FLI import FLI
import numpy as np
import xarray as xr
from os.path import join

In [2]:
%run thr640.py
%run imaging_tool.py

In [3]:
logger = logger
logger.setLevel(50) # 10 - DEBUG 50 - CRITICAL 20 - INFO
camera = FLI()
step_motor = THR640() # Can only have one instance.

SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)

In [5]:
basepath = r'\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration'
os.path.exists(basepath)

True

In [6]:
join(basepath,'a.nc')

'\\\\10.249.254.51\\Public\\Kuzmin\\Experiments-Leprecon\\Spectrometer\\20210129_yun\\Wavelength_calibration\\a.nc'

In [7]:
exposure = 5 * 1e3 # ms
vbin = 1
attributes = {
    "temperature": camera.getTemperature(),
    "device_status": camera.getDeviceStatus(),
    "exposure": exposure,
    "frame_type": "light",
    "motor_coordinate":"",
    "slit_width": "0.3mm",
    "dennryu": 1
}

#R = [100000 + i*5000 for i in range(101)]
#R = [630000 + i*5000 for i in range(4)]
#R = [380000 + i*5000 for i in range(25)]
#R = [500000 + i*5000 for i in range(5)]
#R = [300000 + i*5000 for i in range(101)]
# R = [430000 + i*5000 for i in range(8)] #fulcher-alpha
# R = [430000] # 
#R = [700000 + i*10000 for i in range(10)]
#R = [740000 + i*5000 for i in range(5)]
R = [300000 + i*5000 for i in range(101)]

for r in log_progress(R):
    step_motor.goto(r)
    step_motor.waitUntilReady()
    attributes['motor_coordinate'] = r
    
    camera.setExposureTime(exposure)
    camera.setVBin(vbin)
    camera.setImageArea(10,0,2058,512//vbin)
    time.sleep(.5)
    camera.exposeFrame()
    time.sleep(.1)
    # load image
    image_data = camera.grabFrame(out=np.empty((512//vbin,2048), np.uint16))
    # convert to xarray
    data = xr.DataArray(
        image_data, dims=["y", "x"], coords={"image_counter": 0}, attrs=attributes
    )
    # save data as NetCDF file
    filepath = join(basepath,f'H2_standard_{r}.nc')
    print(filepath)
    data.to_netcdf(filepath)
print('Done.')

\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_300000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_305000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_310000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_315000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_320000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_325000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_330000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_335000.nc
\\10.249.254.51\Public\K

\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_640000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_645000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_650000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_655000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_660000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_665000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_670000.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210129_yun\Wavelength_calibration\H2_standard_675000.nc
\\10.249.254.51\Public\K